In [1]:
import json
import numpy as np
from sys import argv
import sys
import re
#import os # this is use to change to current work directory
#os.chdir("/".join(sys.argv[0].split("/")[:-1]))

import gensim.downloader as api
from gensim.utils import tokenize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings # ignore warnings
warnings.filterwarnings("ignore")

In [2]:
sys.stdout = open('./a4_Dong_111658846_OUTPUT.txt', 'w',encoding='utf-8')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7eu6eY3aq8n2N_0I3Tq3rbdrsplnKHG1Gn-ARXfC_gh1RKYf-LDh0


In [4]:
import os
os.chdir("/content/gdrive/My Drive/CSE354/")

In [5]:
class DOC_RNN(nn.Module):

    def __init__(self, embedding_dim, lstm_hidden_dim, number_of_labels):
        #Initialize all of the layers the forward method will use:
        super(DOC_RNN, self).__init__()

        # Input already has tokens mapped to vectors: no embedding layer needed.
        # For getting LSTM hidden states from embedding vectors:
        self.gru = nn.GRU(embedding_dim, lstm_hidden_dim)

        # Transforms the embedding to a vector of length number_of_tags
        self.linearClassifier = nn.Linear(lstm_hidden_dim, number_of_labels)

    def forward(self, X):
        #Defines how the LSTM will run:
        #X: a list of FloatTensors: [tensor([...]), tensor([...]), ...]
        #   each tensor contains a matrix of input embeddings

        doc_vecs = [] #will hold the final hidden state of each document of X.
        for doc in X: #doc is a FloatTensor of all input embeddings for a single sequence
            s, _ = self.gru(doc.unsqueeze(1))
            #s is now the outputs for all words in the doc
            doc_vecs.append(s[-1]) #represent the doc as the the final word output

        #turn the list of document vectors into a matrix (tensor) of size: num_docs x lstm_hidden_dim
        doc_vecs = torch.stack(doc_vecs).squeeze(1)
        #obtain scores for each doc
        doc_vecs = self.linearClassifier(doc_vecs)
        #change the scores to probabilities1
        yprobs = F.softmax(doc_vecs)
        return yprobs


In [6]:
class GRUModel(nn.Module): # not used, please ignore
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(GRUModel, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        # (seq_len, batch, input_size)#n_layers是GRU的层数
        self.GRU = nn.GRU(in_dim, hidden_dim, n_layer, batch_first=True)
        self.linear = nn.Linear(hidden_dim, n_class)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.GRU(x)
        out = out[:, -1, :]
        out = self.linear(out)
        out = self.sigmoid(out)
        return out

In [7]:
data = []
test = []

with open("./music_QA_train.json", 'r') as infile:
    data = json.load(infile)
with open("./music_QA_dev.json", 'r') as infile:
    test = json.load(infile)
word_embs = api.load('glove-wiki-gigaword-50')

for record in data:
    record['label'] = 1 if record['label'] else 0
    record['question_toks'] = list(tokenize(record['question'], lowercase=True))
    record['passage_toks'] = list(tokenize(record['passage'], lowercase=True))
    for i in range(len(record['question_toks'])):
        try:
            record['question_toks'][i] = word_embs[record['question_toks'][i]]
        except Exception:
            record['question_toks'][i] = word_embs['unk']
    for i in range(len(record['passage_toks'])):
        try:
            record['passage_toks'][i] = word_embs[record['passage_toks'][i]]
        except Exception:
            record['passage_toks'][i] = word_embs['unk']

for record in test:
    record['label'] = 1 if record['label'] else 0
    record['question_toks'] = list(tokenize(record['question'], lowercase=True))
    record['passage_toks'] = list(tokenize(record['passage'], lowercase=True))
    for i in range(len(record['question_toks'])):
        try:
            record['question_toks'][i] = word_embs[record['question_toks'][i]]
        except Exception:
            record['question_toks'][i] = word_embs['unk']
    for i in range(len(record['passage_toks'])):
        try:
            record['passage_toks'][i] = word_embs[record['passage_toks'][i]]
        except Exception:
            record['passage_toks'][i] = word_embs['unk']

In [8]:
net = DOC_RNN(50,30,2).cuda()
#net = GRUModel(50,3,1,2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.5)

In [9]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0

    inputs = [torch.tensor(i['question_toks']).cuda() for i in data]
    labels = torch.tensor([i['label'] for i in data]).cuda()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 50))
    running_loss = 0.0

print('Finished Training')



In [10]:
inputTest = [torch.tensor(i['question_toks']).cuda() for i in test]
labelTest = np.array([i['label'] for i in test])
yPred = net(inputs).cpu().detach().numpy()

result = []
for i in yPred:
    if i[0]>=i[1]:
        result.append(0)
    else:
        result.append(1)

count = 0
for i in range(labelTest.size):
    if result[i] == labelTest[i]:
        count+=1

print("Accuracy rate is:",count/labelTest.size)

---

In [ ]:
 #!pip install transformers

     |████████████████████████████████| 2.1MB 11.2MB/s 
     |████████████████████████████████| 3.3MB 40.2MB/s 
     |████████████████████████████████| 901kB 36.0MB/s 


In [ ]:
 #!pip install datasets

     |████████████████████████████████| 225kB 11.5MB/s 
     |████████████████████████████████| 245kB 16.7MB/s 
     |████████████████████████████████| 112kB 15.9MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model =  AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) # distilbert-base-uncased
model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) # passage model
model3 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) # both

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
train_dataset = Dataset.from_dict({k: [d[k] for d in data] for k in data[0]})
dev_dataset = Dataset.from_dict({k: [d[k] for d in test] for k in test[0]})

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["question"], truncation=True,padding=True)

encoded_train = train_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)
encoded_test = dev_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)

In [ ]:
columns_to_return = ['input_ids', 'label', 'attention_mask']
encoded_train.set_format(type='torch', columns=columns_to_return)
encoded_test.set_format(type='torch', columns=columns_to_return)

In [ ]:
args = TrainingArguments(
    f"question",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.629673,0.605300,140.430000
2,No log,0.605008,0.584700,145.377000
3,No log,0.601476,0.595500,142.735000


TrainOutput(global_step=81, training_loss=0.6367691887749566, metrics={'train_runtime': 31.6183, 'train_samples_per_second': 2.562, 'total_flos': 15688806419844.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': -1892352, 'init_mem_gpu_alloc_delta': 439072256, 'init_mem_cpu_peaked_delta': 1892352, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 4829184, 'train_mem_gpu_alloc_delta': 1321008128, 'train_mem_cpu_peaked_delta': 4096, 'train_mem_gpu_peaked_delta': 211184640})

In [ ]:
# model.save_pretrained('./model/')

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.60147625207901,
 'eval_mem_cpu_alloc_delta': -4096,
 'eval_mem_cpu_peaked_delta': 4096,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 9202688,
 'eval_runtime': 0.5971,
 'eval_samples_per_second': 142.363}

In [ ]:
pred = trainer.predict(encoded_test)
lst = [0 if i[0]>i[1] else 1 for i in pred.predictions]
count=0
for i in range(len(lst)):
  if lst[i] == pred.label_ids[i]:
    count+=1
print("Accuraccy rate of question model is:",count/len(lst))

---

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["passage"], truncation=True,padding=True)

encoded_train = train_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)
encoded_test = dev_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)

In [ ]:
columns_to_return = ['input_ids', 'label', 'attention_mask']
encoded_train.set_format(type='torch', columns=columns_to_return)
encoded_test.set_format(type='torch', columns=columns_to_return)

In [ ]:
args2 = TrainingArguments(
    f"passage",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer2 = Trainer(
    model2,
    args2,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.649209,3.475900,24.454000
2,No log,0.641420,3.465200,24.530000
3,No log,0.636313,3.474300,24.466000


TrainOutput(global_step=159, training_loss=0.6332615786378488, metrics={'train_runtime': 262.9532, 'train_samples_per_second': 0.605, 'total_flos': 395523067110804.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 439072256, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 0, 'train_mem_gpu_alloc_delta': 1380382720, 'train_mem_cpu_peaked_delta': 8192, 'train_mem_gpu_peaked_delta': 5876984832})

In [ ]:
# model2.save_pretrained('./model2/')

In [ ]:
trainer2.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.6363130211830139,
 'eval_mem_cpu_alloc_delta': -4096,
 'eval_mem_cpu_peaked_delta': 4096,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 123499008,
 'eval_runtime': 3.456,
 'eval_samples_per_second': 24.595}

In [ ]:
pred1 = trainer2.predict(encoded_test)
lst = [0 if i[0]>i[1] else 1 for i in pred1.predictions]
count=0
for i in range(len(lst)):
  if lst[i] == pred1.label_ids[i]:
    count+=1
print("Accuraccy rate of passage model is:",count/len(lst))

---

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["question"],examples["passage"], truncation=True,padding=True,add_special_tokens=True)

encoded_train = train_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)
encoded_test = dev_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)

In [ ]:
columns_to_return = ['input_ids', 'label', 'attention_mask']
encoded_train.set_format(type='torch', columns=columns_to_return)
encoded_test.set_format(type='torch', columns=columns_to_return)

In [ ]:
args3 = TrainingArguments(
    f"passage",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer3 = Trainer(
    model3,
    args3,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer3.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.661018,3.822200,22.238000
2,No log,0.611430,3.829500,22.196000
3,0.636700,0.711569,3.848000,22.089000


TrainOutput(global_step=630, training_loss=0.593750006055075, metrics={'train_runtime': 530.7304, 'train_samples_per_second': 1.187, 'total_flos': 404606060301240.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -483041280, 'train_mem_gpu_alloc_delta': 896167424, 'train_mem_cpu_peaked_delta': 484261888, 'train_mem_gpu_peaked_delta': 1534983168})

In [ ]:
trainer3.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.7115694284439087,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 48338432,
 'eval_runtime': 3.8927,
 'eval_samples_per_second': 21.836}

In [ ]:
pred2 = trainer3.predict(encoded_test)
lst = [0 if i[0]>i[1] else 1 for i in pred2.predictions]
count=0
for i in range(len(lst)):
  if lst[i] == pred2.label_ids[i]:
    count+=1
print("Accuraccy rate of full question-passage model is:",count/len(lst))

---

In [ ]:
question1 = "Is weather going to be change at constant rate?"
question2 = "Am I the most handsome man on the world?"
question3 = "Could you please say yes?"
q1 = tokenizer(question1, truncation=True,padding=True)
q2 = tokenizer(question2, truncation=True,padding=True)
q3 = tokenizer(question3, truncation=True,padding=True)

In [ ]:
toTest = {
    'label':[0,0,0],
    'input_ids':[q1['input_ids'],q2['input_ids'],q3['input_ids']],
    'attention_mask':[q1['attention_mask'],q2['attention_mask'],q3['attention_mask']]
}
que3 = Dataset.from_dict(toTest)

In [ ]:
prediction = ["True" if i[0]>i[1] else "False" for i in trainer3.predict(que3).predictions]

In [ ]:
print("----------------------------------------------------------------------")
print("Three Questions:\n")
print("Question 1:",question1)
print("Answer 1:",prediction[0])
print("Question 2:",question2)
print("Answer 2:",prediction[1])
print("Question 3:",question3)
print("Answer 3:",prediction[2])
print("----------------------------------------------------------------------")

---

In [ ]:
model4 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=2) # fine-tune another large model

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["question"],examples["passage"], truncation=True,padding=True,add_special_tokens=True)

encoded_train = train_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)
encoded_test = dev_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)

In [ ]:
columns_to_return = ['input_ids', 'label', 'attention_mask']
encoded_train.set_format(type='torch', columns=columns_to_return)
encoded_test.set_format(type='torch', columns=columns_to_return)

In [ ]:
args4 = TrainingArguments(
    f"passage",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer4 = Trainer(
    model4,
    args4,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
)# Taken from Hugging Face Sequence Classification Tutorial

In [ ]:
trainer4.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.648278,3.866100,21.986000
2,No log,0.664938,3.869000,21.970000
3,0.681800,0.644400,3.844500,22.109000


TrainOutput(global_step=630, training_loss=0.6813916645352802, metrics={'train_runtime': 498.8154, 'train_samples_per_second': 1.263, 'total_flos': 618484707643320.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 671755264, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -801206272, 'train_mem_gpu_alloc_delta': 2048315904, 'train_mem_cpu_peaked_delta': 808161280, 'train_mem_gpu_peaked_delta': 1534617088})

In [ ]:
trainer4.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.6444003582000732,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 47918592,
 'eval_runtime': 3.8594,
 'eval_samples_per_second': 22.024}

In [ ]:
pred4 = trainer4.predict(encoded_test)
lst = [0 if i[0]>i[1] else 1 for i in pred4.predictions]
count=0
for i in range(len(lst)):
  if lst[i] == pred4.label_ids[i]:
    count+=1
print("Accuraccy rate of passage model is:",count/len(lst))

---

In [ ]:
print("Now making prediction on Test set...")

In [ ]:
final = []
with open("./music_QA_test.json", 'r') as infile:
    final = json.load(infile)
final_dataset = Dataset.from_dict({k: [d[k] for d in final] for k in final[0]})

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["question"],examples["passage"], truncation=True,padding=True,add_special_tokens=True)

final_data = final_dataset.map(preprocess_function, 
batched=True,load_from_cache_file=False)
columns_to_return = ['input_ids', 'attention_mask']
final_data.set_format(type='torch', columns=columns_to_return)

In [ ]:
pred_final = trainer.predict(final_data).predictions

In [ ]:
pair = [0 if i[0]>i[1] else 1for i in pred_final]

In [ ]:
import pandas as pd
df = pd.DataFrame(pair,columns=['label'], index=[i['idx'] for i in final_dataset])

In [ ]:
df.reset_index().rename(columns={'index':'idx'}).to_csv("./Dong_Kaggle_OUTPUT.csv", index=False, sep=',')

In [ ]:
print("Done. Please check .csv")